In [20]:
# Full, clean Python script with 20 kW motor and final spread/output table

import numpy as np
import pandas as pd

# --- 1. Sunshine profile per hour (from Irrigation Tech!B33:B45) ---
sunshine_percent = np.array([0, 0.03, 0.32, 0.5, 0.75, 0.9, 1.0, 0.99, 0.85, 0.65, 0.3, 0.01, 0])

# --- 2. External Inputs ---
solar_kw = 15.54
motor_kw = 15  # updated for this run
loss_factor = 0.02
drive_min_pct = 0.35

# --- 3. Monthly total pumping hours from Excel (L27:X27) ---
monthly_pump_hours = {
    "Jan": 7.52688172, "Feb": 14.2857143, "Mar": 4.30107527, "Apr": 2.222222,
    "May": 7.826002, "Jun": 0.222222, "Jul": 2.0, "Aug": 3.225806,
    "Sep": 11.11111, "Oct": 11.82796, "Nov": 14.44444, "Dec": 16.11111
}

# --- 4. Drive and pump output calculations ---
sizing_factor = solar_kw / motor_kw
drive_output = np.minimum(sunshine_percent * sizing_factor, 1.0)
F = np.full_like(drive_output, motor_kw * 1000)
G = np.where(drive_output >= drive_min_pct, drive_output * F, 0)
H = np.where(G > 0, G * (1 - loss_factor), 0)
I = H / F  # Per-hour pump output fraction (relative to motor)

# --- 5. Excel-matching distribution logic for spread ---
def build_excel_matched_distribution_fixed(total_hours):
    spread = np.zeros(13)
    center = 6  # 12:00
    if total_hours <= 1:
        spread[center] = total_hours
        return spread
    spread[center] = 1
    used = 1
    offset = 1
    while used + 2 <= total_hours and offset <= 6:
        spread[center - offset] = 1
        spread[center + offset] = 1
        used += 2
        offset += 1
    remaining = total_hours - used
    if remaining > 0 and offset <= 6:
        spread[center - offset] = remaining / 2
        spread[center + offset] = remaining / 2
    return spread

# --- 6. Calculate monthly spread and output kWh/day ---
results = {}
spread_matrix = {}

for month, hours in monthly_pump_hours.items():
    spread = build_excel_matched_distribution_fixed(hours)
    output_kwh = float(np.sum(spread * I))
    results[month] = output_kwh
    spread_matrix[month] = spread

# --- 7. Prepare final output table ---
spread_df = pd.DataFrame(spread_matrix, index=[f"{h:02d}:00" for h in range(6, 19)])
spread_df.loc["Total Hours (L27:X27)"] = monthly_pump_hours
spread_df.loc["Daily Output kWh (L43:X43)"] = results

spread_df


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
06:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055555,0.413980,1.000000,1.000000
07:00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
08:00,0.263441,1.000000,0.000000,0.000000,0.413001,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
09:00,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
10:00,1.000000,1.000000,0.650538,0.000000,1.000000,0.000000,0.000000,0.112903,1.000000,1.000000,1.000000,1.000000
11:00,1.000000,1.000000,1.000000,0.611111,1.000000,0.000000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000
12:00,1.000000,1.000000,1.000000,1.000000,1.000000,0.222222,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
13:00,1.000000,1.000000,1.000000,0.611111,1.000000,0.000000,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000
14:00,1.000000,1.000000,0.650538,0.000000,1.000000,0.000000,0.000000,0.112903,1.000000,1.000000,1.000000,1.000000
15:00,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
